# Running CEDS Scenarios

In this notebook we document how to process and run data from the CEDS database.

In [1]:
%matplotlib inline

from os import listdir
from os.path import join, dirname
from pprint import pprint

import pandas as pd
import pyam
import pymagicc
from pymagicc.io import MAGICCData
import matplotlib.pyplot as plt
plt.style.use('bmh') 

import expectexception

<IPython.core.display.Javascript object>

In [3]:
TEST_DATA_PATH = join("..", "tests", "test_data")

In [9]:
pyam_df = pyam.IamDataFrame(data=join(TEST_DATA_PATH, "ceds-format-example.csv"), encoding='utf-8')

INFO:root:Reading `../tests/test_data/ceds-format-example.csv`


In [11]:
raw_df.head()

,model,scenario,region,variable,unit,year,value
0,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5ASIA,Emissions|BC,Mt BC/yr,2015,4
91,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5LAM,Emissions|BC,Mt BC/yr,2015,4
174,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5MAF,Emissions|BC,Mt BC/yr,2015,6
257,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5OECD,Emissions|BC,Mt BC/yr,2015,8
348,MODEL-NAME-HYPHENS,SCENARIO-NAME-HYPHENS,R5REF,Emissions|BC,Mt BC/yr,2015,3


In [ ]:
def read_ceds_csv(filepath=None, filename=None):

In [49]:
raw_df = pyam_df.data
reindex_df = raw_df.set_index(["model", "scenario", "region", "variable", "unit", "year"]).unstack().T
reindex_df.index = reindex_df.index.get_level_values("year")
reindex_df.index.name = "YEAR"
models = reindex_df.columns.get_level_values("model")
scenarios = reindex_df.columns.get_level_values("scenario")
regions = reindex_df.columns.get_level_values("region")
variables = reindex_df.columns.get_level_values("variable")
units = reindex_df.columns.get_level_values("unit")
todos = ["SET"] * len(units)
reindex_df.columns = pd.MultiIndex.from_arrays(
    [models, scenarios, variables, todos, units, regions],
    names=("MODEL", "SCENARIO", "VARIABLE", "TODO", "UNITS", "REGION"),
)
reindex_df

MODEL       MODEL-NAME-HYPHENS                                          \
SCENARIO SCENARIO-NAME-HYPHENS                                           
VARIABLE          Emissions|BC Emissions|BC|Agricultural Waste Burning   
TODO                       SET                                     SET   
UNITS                 Mt BC/yr                                Mt BC/yr   
REGION                  R5ASIA                                  R5ASIA   
YEAR                                                                     
2015                         4                                       6   
2020                         3                                       9   
2030                         6                                      10   
2040                         3                                       3   
2050                         0                                       3   
2060                         0                                       2   
2070                         4                                       6   
2080                         7                                       6   
2090                         8                                       1   
2100                         9                                       1   

MODEL                                                            \
SCENARIO                                                          
VARIABLE Emissions|BC|Energy Sector Emissions|BC|Forest Burning   
TODO                            SET                         SET   
UNITS                      Mt BC/yr                    Mt BC/yr   
REGION                       R5ASIA                      R5ASIA   
YEAR                                                              
2015                              0                           3   
2020                             10                           1   
2030                              7                           7   
2040                              1                          10   
2050                              7                           7   
2060                              2                           1   
2070                              7                           4   
2080                             10                           7   
2090                             10                           5   
2100                              5                           8   

MODEL                                                                   \
SCENARIO                                                                 
VARIABLE Emissions|BC|Grassland Burning Emissions|BC|Industrial Sector   
TODO                                SET                            SET   
UNITS                          Mt BC/yr                       Mt BC/yr   
REGION                           R5ASIA                         R5ASIA   
YEAR                                                                     
2015                                  8                              0   
2020                                  5                              8   
2030                                  1                              9   
2040                                  2                              3   
2050                                  5                              8   
2060                                  9                              7   
2070                                  1                              7   
2080                                  2                              0   
2090                                  0                              7   
2100                                  2                              9   

MODEL                                                                         \
SCENARIO                                                                       
VARIABLE Emissions|BC|Peat Burning Emissions|BC|Residential Commercial Other   
TODO                           SET                                       SET   
UNITS                 

In [24]:
# adf = raw_df.set_index("model").stack()
# adf = pd.DataFrame(adf)
# adf

0
model                                             
MODEL-NAME-HYPHENS scenario  SCENARIO-NAME-HYPHENS
                   region                   R5ASIA
                   variable           Emissions|BC
                   unit                   Mt BC/yr
                   year                       2015
                   value                         4
                   scenario  SCENARIO-NAME-HYPHENS
                   region                    R5LAM
                   variable           Emissions|BC
                   unit                   Mt BC/yr
                   year                       2015
                   value                         4
                   scenario  SCENARIO-NAME-HYPHENS
                   region                    R5MAF
                   variable           Emissions|BC
                   unit                   Mt BC/yr
                   year                       2015
                   value                         6
                   scenario  SCENARIO-NAME-HYPHENS
                   region                   R5OECD
                   variable           Emissions|BC
                   unit                   Mt BC/yr
                   year                       2015
                   value                         8
                   scenario  SCENARIO-NAME-HYPHENS
                   region                    R5REF
                   variable           Emissions|BC
                   unit                   Mt BC/yr
                   year                       2015
                   value                         3
...                                            ...
                   scenario  SCENARIO-NAME-HYPHENS
                   region                    R5LAM
                   variable    Emissions|VOC|Waste
                   unit                  Mt VOC/yr
                   year                       2100
                   value                         3
                   scenario  SCENARIO-NAME-HYPHENS
                   region                    R5MAF
                   variable    Emissions|VOC|Waste
                   unit                  Mt VOC/yr
                   year                       2100
                   value                        10
                   scenario  SCENARIO-NAME-HYPHENS
                   region                   R5OECD
                   variable    Emissions|VOC|Waste
                   unit                  Mt VOC/yr
                   year                       2100
                   value                         8
                   scenario  SCENARIO-NAME-HYPHENS
                   region                    R5REF
                   variable    Emissions|VOC|Waste
                   unit                  Mt VOC/yr
                   year                       2100
                   value                         4
                   scenario  SCENARIO-NAME-HYPHENS
                   region                    World
                   variable    Emissions|VOC|Waste
                   unit                  Mt VOC/yr
                   year                       2100
                   value                        10

[34140 rows x 1 columns]